In [1]:
from lib.data_util import *
from lib.documentation import *
from lib.data_models import *
import matplotlib.pyplot as plt

### Pre-process data

In [ ]:
df = read_data("data/statcast_pitch_swing_data_20240402_20240630.arrow")
build_description_table(df, "glossary/data_definitions.md", "glossary/data_dictionary.md")

polars.config.Config

In [3]:
pitcher_tree = tree(df, "pitcher", Pitcher)
batter_tree = tree(df, "batter", Batter)

In [ ]:
plt.figure(figsize=(100, 15))
mean_bat_speed = df.filter(pl.col("bat_speed").is_not_null()).group_by("batter").agg(pl.col("bat_speed").mean().alias("mean_bat_speed")).sort("mean_bat_speed")
plt.bar(mean_bat_speed["batter"].cast(pl.String), mean_bat_speed["mean_bat_speed"])
plt.xticks(rotation=90)
plt.savefig("visualizations/mean_bat_speed.png")
